In [135]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.sparse.linalg as sla

Nota che la matrice A è tale che, se presa verticalmente, i valori sommati sono pari a 1, mentre nella versione trasposta, ovviamente, la somma sulle righe fa 1

In [78]:
A = np.array([
    [0.8, 0.2, 0.1],
    [0.1, 0.7, 0.3],
    [0.1, 0.1, 0.6]
])
t_curr = np.array([0.4, 0.24, 0.36])


# you can do A @ t_curr.T
# or t_curr @ A.T

A @ t_curr.T

array([0.404, 0.316, 0.28 ])

In [94]:
# la funzione presume che A sia tale che la somma delle colonne sia 1
def convergence(A, t_curr, eps=1e-20):
    assert np.allclose(A.sum(axis=0), 1)
    t = t_curr.copy()
    t_next = t @ A.T
    while (abs(t_next - t)).sum() > eps:
        t = t_next
        t_next = t @ A.T
    return t_next

def get_nth_step(A, t_curr, n):
    t = t_curr.copy()
    for _ in range(n):
        t = t @ A.T
    return t

print(convergence(A, t_curr))

print(get_nth_step(A, t_curr, 4))

[0.45 0.35 0.2 ]
[0.432784 0.357216 0.21    ]


Usiamo il metodo alternativo che fa uso della potenza della matrice di transizione A

In [74]:
def get_nth_state_alternative(A, t_curr, n):
    A_n = np.linalg.matrix_power(A, n)
    return t_curr @ A_n.T

print(get_nth_state_alternative(A, t_curr, 4))

[0.432784 0.357216 0.21    ]


In [75]:
A = np.array([
    [0.92, .03, .02, .01],
    [.02, .94, .02, .01],
    [.01, .01, .90, .01],
    [0.05, .02, .06, .97]
])
# row  : to 
# col : from 

t_0 = np.array([0.40, 0.32, 0.18, 0.10])

convergence(A, t_0, eps=1e-20)


array([0.1609538 , 0.17883756, 0.09090909, 0.56929955])

In [95]:
A = np.array([
    [0.8, 0.1],
    [0.2, 0.9]
])
print(convergence(A,[1,0]))
print(convergence(A,[0.5,0.5]))
print(convergence(A,[0,1]))

[0.33333333 0.66666667]
[0.33333333 0.66666667]
[0.33333333 0.66666667]


Indipendentemente dallo stato iniziale lo stato finale è sempre lo stesso

In [174]:
def convergence_stable_distr_matrix(A,t_0,eps=1e-20):
    assert np.allclose(A.sum(axis=0), 1)
    A_cur = A.copy()
    A_next = A_cur @ A
    while (abs(A_next - A_cur)).sum() > eps:
        A_cur = A_next
        A_next = A_cur @ A

    return t_0 @ A_next.T

# è stocastica
A = np.array([
    [0.5, 0.25],
    [0.5, 0.75]
])

print("Using stable distribution matrix")
print(convergence_stable_distr_matrix(A,[.4,.6]))
print()
print("Vanilla method")
print(convergence(A,[.4,.6]))

Using stable distribution matrix
[0.33333333 0.66666667]

Vanilla method
[0.33333333 0.66666667]


I risultati sono identici!

Come facciamo a trovare la stable distribution matrix?

E' la soluzione di

A x = x

che sarebbe l'autovettore associato all'autovalore 1

In [181]:
def find_steady_state(A):
    assert np.allclose(A.sum(axis=0), 1)
    eval,evect = np.linalg.eig(A)
    index = np.argmin(np.abs(eval - 1))
    steady_state = evect[:, index].real
    steady_state /= np.sum(steady_state)
    return steady_state
def find_stable_distribution_matrix(A):
    ss = find_steady_state(A)
    return np.full_like(A,ss).T
    
A = np.array([
    [0.9, 0.5],
    [0.1, 0.5]
])

print(find_steady_state(A))
print()
print(find_stable_distribution_matrix(A))
    

[0.83333333 0.16666667]

[[0.83333333 0.83333333]
 [0.16666667 0.16666667]]


In [182]:
A = np.array([
    [0.95, 0.5],
    [0.05, 0.5]
])

find_steady_state(A)

array([0.90909091, 0.09090909])

In [185]:
A = np.array([
    [0.8,0.2,0.3],
    [0.1,0.7,0.1],
    [0.1,0.1,0.6]
])

find_steady_state(A)

array([0.55, 0.25, 0.2 ])

In [187]:
A = np.array([
    [0.9,0.4],
    [0.1,0.6]
])

find_steady_state(A)
    

array([0.8, 0.2])

Absorbing Markov Chain


In [189]:
A = np.array([
    [1, 0.2, 0.2],
    [0, 0.6, 0.1],
    [0, 0.2, 0.7]
])

print(find_steady_state(A))
print()
print(A.T)


[1. 0. 0.]

[[1.  0.  0. ]
 [0.2 0.6 0.2]
 [0.2 0.1 0.7]]


In [191]:
A = np.array([
    [1, 0, 0.2, 0.1],
    [0, 1, 0.1, 0.2],
    [0, 0, 0.5, 0.1],
    [0, 0, 0.2, 0.6]
])

find_steady_state(A)

array([1., 0., 0., 0.])